In [2]:
import numpy as np
import tensorflow as tf
import keras
import argparse
import h5py
from keras import optimizers
from keras import backend as K
from keras.losses import categorical_crossentropy as logloss
from keras.metrics import categorical_accuracy, top_k_categorical_accuracy
from keras.datasets import cifar10, cifar100
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import Iterator
# from keras.engine.topology import Input, Container
from keras.engine.training import Model
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, advanced_activations, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, Convolution2D, pooling, Lambda, concatenate
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.models import load_model as keras_load_model
from keras.callbacks import Callback
import pytz
from datetime import datetime
import os
from keras.utils import multi_gpu_model
from tensorflow.python.client import device_lib

In [3]:
import numpy as np
import keras
from keras.models import  Model,load_model
from keras.layers import Dense,Dropout,Conv2D,MaxPooling2D,Input,GlobalAveragePooling2D,BatchNormalization
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler
from keras.datasets import cifar10
from keras.utils.np_utils import to_categorical
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
import pickle

In [4]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [5]:
# データ次元確認
print('x_train.shape:', x_train.shape)
print('x_test.shape:', x_test.shape)
print('y_train.shape:', y_train.shape)
print('y_test.shape:', y_test.shape)

x_train.shape: (50000, 32, 32, 3)
x_test.shape: (10000, 32, 32, 3)
y_train.shape: (50000, 1)
y_test.shape: (10000, 1)


In [6]:
from keras.utils.np_utils import to_categorical
# 正規化
x_train = x_train/255.
x_test = x_test/255.

# one hot ベクトル化
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

## 蒸留モデル

In [7]:
def knowledge_distillation_loss(input_distillation):
    y_pred, y_true, y_soft, y_pred_soft = input_distillation
    return (1 - args.lambda_const) * logloss(y_true, y_pred) + \
           args.lambda_const * args.temperature * args.temperature * logloss(y_soft, y_pred_soft)


class MyIterator(object):
    def __init__(self, iterator_org):
        self.iterator = iterator_org

    def __iter__(self):
        return self

    def __next__(self):
        tmp = next(self.iterator)
        return [tmp[0], tmp[1]], tmp[1]

In [8]:
class BornAgainModel(object):
    def __init__(self, teacher_model):
        self.train_model, self.born_again_model = None, None
        self.temperature = args.temperature
        self.teacher_model = keras_load_model(teacher_model)
        for i in range(len(self.teacher_model.layers)):
            self.teacher_model.layers[i].trainable = False
        self.teacher_model.compile(optimizer="adam", loss="categorical_crossentropy")
        self.train_model, self.born_again_model = self.prepare()
        self.train_model = convert_gpu_model(self.train_model)

    def prepare(self):
#         self.teacher_model.summary()
        self.teacher_model.layers.pop()
        input_layer = self.teacher_model.input
        teacher_logits = self.teacher_model.layers[-1].output
        teacher_logits_T = Lambda(lambda x: x / self.temperature)(teacher_logits)
        teacher_probabilities_T = Activation('softmax', name='softmax1_')(teacher_logits_T)

        """生徒モデル作成部分
        """
        # 1/4に設定
        x = Conv2D(16,(3,3),padding = "SAME",activation= "relu")(input_layer)
        x = Conv2D(16,(3,3),padding = "SAME",activation= "relu")(x)
        x = Dropout(0.25)(x)
        x = MaxPooling2D()(x)

        x = Conv2D(32,(3,3),padding = "SAME",activation= "relu")(x)
        x = Conv2D(32,(3,3),padding = "SAME",activation= "relu")(x)
        x = Dropout(0.25)(x)
        x = MaxPooling2D()(x)

        x = Conv2D(64,(3,3),padding = "SAME",activation= "relu")(x)
        x = Conv2D(64,(3,3),padding = "SAME",activation= "relu")(x)
        x = GlobalAveragePooling2D()(x)

        # 1024->512
        x = Dense(512,activation = "relu")(x)
        x = Dropout(0.25)(x)
        """生徒モデル作成部分
        """
        
        logits = Dense(num_classes, activation=None, name='dense2')(x)
        output_softmax = Activation('softmax', name='output_softmax')(logits)
        logits_T = Lambda(lambda x: x / self.temperature, name='logits')(logits)
        probabilities_T = Activation('softmax', name='probabilities')(logits_T)

        with tf.device('/cpu:0'):
            born_again_model = Model(inputs=input_layer, outputs=output_softmax)
            input_true = Input(name='input_true', shape=[None], dtype='float32')
        output_loss = Lambda(knowledge_distillation_loss, output_shape=(1,), name='kd_')(
            [output_softmax, input_true, teacher_probabilities_T, probabilities_T]
        )
        inputs = [input_layer, input_true]

        with tf.device('/cpu:0'):
            train_model = Model(inputs=inputs, outputs=output_loss)

        return train_model, born_again_model

    def evaluate(self):
        y_pred = self.born_again_model.predict(x_test)
        acc = 0
        for i in range(y_pred.shape[0]):
            if np.argmax(y_pred[i][:num_classes]) == np.argmax(y_test[i]):
                acc = acc + 1

        return acc / y_pred.shape[0]
    
def convert_gpu_model(org_model: Model) -> Model:
    gpu_count = len(device_lib.list_local_devices()) - 1
    if gpu_count > 1:
        train_model = multi_gpu_model(org_model, gpu_count)
    else:
        train_model = org_model
    return train_model

In [9]:
num_classes = 10 # cifar10用

In [10]:
import easydict
args = easydict.EasyDict({
        "temperature": 10.0,
        "lambda_const": 0.9,
        "teacher_model_path": './models/cifar10_tf114_functional_api_ver_200604.h5'
})


distilmodel = BornAgainModel(args.teacher_model_path)
print()
distilmodel.born_again_model.summary()





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 16)        448       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 16)        2320      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 16)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 16)        0         
_________________________________________________________

In [11]:
distilmodel.born_again_model.compile(loss = "categorical_crossentropy",optimizer = Adam(), metrics = ["accuracy"])
distilmodel.born_again_model.fit(x_train,
                              y_train,
                              batch_size=32,
                              epochs=20,
                              validation_split=0.2)

Train on 40000 samples, validate on 10000 samples
Epoch 1/20
40000/40000 [==============================] - 9s 219us/step - loss: 1.7772 - acc: 0.3223 - val_loss: 1.5927 - val_acc: 0.4147
Epoch 2/20
40000/40000 [==============================] - 7s 169us/step - loss: 1.3815 - acc: 0.4872 - val_loss: 1.3935 - val_acc: 0.4809
Epoch 3/20
40000/40000 [==============================] - 7s 171us/step - loss: 1.1898 - acc: 0.5619 - val_loss: 1.1520 - val_acc: 0.5950
Epoch 4/20
40000/40000 [==============================] - 7s 171us/step - loss: 1.0753 - acc: 0.6110 - val_loss: 1.0875 - val_acc: 0.6135
Epoch 5/20
40000/40000 [==============================] - 7s 171us/step - loss: 0.9840 - acc: 0.6486 - val_loss: 0.9765 - val_acc: 0.6521
Epoch 6/20
40000/40000 [==============================] - 7s 167us/step - loss: 0.9178 - acc: 0.6733 - val_loss: 1.0104 - val_acc: 0.6395
Epoch 7/20
40000/40000 [==============================] - 7s 171us/step - loss: 0.8599 - acc: 0.6944 - val_loss: 1.0279 - 

In [12]:
# Score trained model.
scores = distilmodel.born_again_model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 0s 50us/step
Test loss: 0.7665595895767212
Test accuracy: 0.7378


In [14]:
import os
# Save model and weights
save_dir = os.path.join(os.getcwd(), 'models')
model_name = 'cifar10_distillation_tf114_functional_api_ver_200604.h5'
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
distilmodel.born_again_model.save(model_path)
print('Saved trained model at %s ' % model_path)

Saved trained model at C:\Users\12810649\Projects\dnn_samples\cifar10\models\cifar10_distillation_tf114_functional_api_ver_200604.h5 
